In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import yfinance as yf


yff = yf.download("AAPL", start="2019-12-19", end="2019-12-19")
print(yff.head())




C:\Users\apurv\AppData\Local\Temp\ipykernel_31856\2323955529.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yff = yf.download("AAPL", start="2019-12-19", end="2019-12-19")
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-12-19 -> 2019-12-19)')


Empty DataFrame
Columns: [(Adj Close, AAPL), (Close, AAPL), (High, AAPL), (Low, AAPL), (Open, AAPL), (Volume, AAPL)]
Index: []
Empty DataFrame
Columns: [(Adj Close, AAPL), (Close, AAPL), (High, AAPL), (Low, AAPL), (Open, AAPL), (Volume, AAPL)]
Index: []


In [ ]:
import os
import time
import pandas as pd
import yfinance as yf

#tried example usage for the yfinance dataset

ticker = "AAPL"
start_date = "2019-12-01"
end_date = "2019-12-31"
data = yf.download(ticker, start=start_date, end=end_date, group_by='ticker')
if not data.empty:
    print(data.head())
    # Save to CSV
    output_dir = "../data/Stock-Market-Data/yf"
    os.makedirs(output_dir, exist_ok=True)
    data.to_csv(os.path.join(output_dir, f"{ticker}.csv"))

Downloaded data for AAPL (20 rows)
Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2019-12-02  63.736343  64.723175  63.565037  64.486719   94487200
2019-12-03  62.599941  62.619240  61.837497  62.324879  114430400
2019-12-04  63.152481  63.531292  62.896725  62.990828   67181600
2019-12-05  64.078972  64.153775  63.391332  63.647087   74424400
2019-12-06  65.316727  65.386700  64.493963  64.537399  106075600


C:\Users\apurv\AppData\Local\Temp\ipykernel_31856\1479711222.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, progress=False)


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Flatten MultiIndex columns
def flatten_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(col).strip() for col in df.columns.values]
    return df

# Preprocess yfinance data for LSTM
def preprocess_data(data, sequence_length=20):
    # Flatten columns if MultiIndex
    data = flatten_columns(data)
    
    # Check if 'Close_AAPL' exists
    if 'Close_AAPL' not in data.columns:
        raise KeyError("'Close_AAPL' column is missing. Available columns: " + ", ".join(data.columns))
    
    # Feature engineering: calculate daily returns
    data['return'] = data['Close_AAPL'].pct_change()
    data = data.dropna()  # Drop rows with NaN values

    # Select features and scale them
    features = ['return', 'Volume_AAPL']
    if not all(col in data.columns for col in features):
        raise KeyError("One or more required columns are missing. Available columns: " + ", ".join(data.columns))
    
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(data[features])

    # Create sequences for LSTM
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predict next return
    
    X, y = np.array(X), np.array(y)
    print(f"Created {len(X)} sequences of shape {X.shape}")
    return X, y, scaler

# Example usage
data = flatten_columns(data)  # Flatten columns before preprocessing
sequence_length = 20
X, y, scaler = preprocess_data(data, sequence_length)


MultiIndex([( 'Close', 'AAPL'),
            (  'High', 'AAPL'),
            (   'Low', 'AAPL'),
            (  'Open', 'AAPL'),
            ('Volume', 'AAPL')],
           names=['Price', 'Ticker'])
